In [1]:
import pandas as pd
import pandas as pd 
import numpy as np
import os 
import sys
sys.path.append('./../../.')
sys.path.append('./../.')
import glob 
from tqdm import tqdm 
import joblib
from joblib import Parallel, delayed
from pandarallel import pandarallel
pandarallel.initialize()
import re
import yaml
from collections import Counter
import pickle
from itertools import combinations
import itertools
import random

DATA_SOURCE = './../generated_data_v1/'
CONFIG = None
DIR_LOC = None
CONFIG = None
CONFIG_FILE = 'config.yaml'
id_col = 'PanjivaRecordID'
non_CoOcc_dict = {}
numeric_col_stats = {}
DATA_PATH = ''

def set_up_config(_DIR = None):
    global DIR
    global CONFIG
    global CONFIG_FILE
    global use_cols
    global freq_bound
    global num_neg_samples_ape
    global save_dir
    global column_value_filters
    global num_neg_samples
    global NUMERIC_COLUMNS
    global id_col
    global DISCRETE_COLUMNS
    global DATA_PATH
    global DATA_SOURCE
    
    with open(CONFIG_FILE) as f:
        CONFIG = yaml.safe_load(f)
    
    DATA_PATH = os.path.join(DATA_SOURCE, DIR)
    if _DIR is not None:
        DIR = _DIR
        CONFIG['DIR'] = _DIR
    else:
        DIR = CONFIG['DIR']

    DIR_LOC = re.sub('[0-9]', '', DIR)
    DATA_PATH = os.path.join(DATA_SOURCE, DIR)
    save_dir =  CONFIG['save_dir']
    
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_dir = os.path.join(
        CONFIG['save_dir'],
        DIR
    )
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
        
    use_cols = CONFIG[DIR]['use_cols']
 
    NUMERIC_COLUMNS = CONFIG[DIR]['numeric_columns']
    _cols = list(use_cols)
    _cols.remove(id_col)
    for nc in NUMERIC_COLUMNS:
        _cols.remove(nc)
        
    DISCRETE_COLUMNS = list(sorted(_cols))
    return 



def fetch_data_sets():
    global DATA_PATH
    df_train = pd.read_csv(os.path.join(DATA_PATH,'train_data.csv'),index_col=None)
    df_test = pd.read_csv(os.path.join(DATA_PATH,'test_data.csv'),index_col=None)
    return df_train, df_test


def hash1(row,c1,c2):
    return str(row[c1]) + '_' + str(row[c2])


def create_coocc_dict(df_train):
    global DISCRETE_COLUMNS
    global non_CoOcc_dict
    df = df_train.copy()
    for pair in combinations(DISCRETE_COLUMNS,2):
        pool = []
        col_1 = pair[0]
        col_2 = pair[1]
        tmp = df.groupby([col_1,col_2]).size().reset_index(name='count')
        
        a = set(df[col_1])
        b = set(df[col_2])
        tmp['_hash'] = tmp.parallel_apply(hash1, axis=1, args=(col_1,col_2,))
        present = list(tmp['_hash'])
        for i, j in itertools.product(a, b):
            pool.append(str(i) + '_' + str(j))
        
        pool = set(pool)
     
        pool = pool.difference(present)
        
        pool = [ _.split('_') for _ in pool ]
        key = col_1 + '_' + col_2
        print(len(pool))
        non_CoOcc_dict[key] = pool
    return 

def perturb_row(row, num_pairs = 1, num_numeric_cols=1 ):
    global DISCRETE_COLUMNS
    global NUMERIC_COLUMNS
    global numeric_col_stats 
    global non_CoOcc_dict
    row = row.copy()
    pert_cols = np.random.choice( DISCRETE_COLUMNS , size= num_pairs*2, replace = False)
    for i in range(1, num_pairs+1):
        cols = sorted(pert_cols[(i-1)*2:(i-1)*2+2])
        cols = sorted(cols)
        col_1 = cols[0]
        col_2 = cols[1]
        key = col_1 + '_' + col_2
        arr =  non_CoOcc_dict[key]
        _idx = random.randint(0, len(arr))
        vals = arr[_idx]
        row[col_1] = vals[0]
        row[col_2] = vals[1]
    
    # Select a numeric column 
    
    numeric_cols = np.random.choice( NUMERIC_COLUMNS , size = num_numeric_cols, replace = False)
    for nc in numeric_cols:
        val = row[nc]
        if val < 0.5:
            val += random.uniform(0.25,0.75) 
        else:
            val -= random.uniform(0.25,0.75) 
        row[nc] = val
    return row

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# =================================================================
# Main code 
# =================================================================

In [4]:
DIR = 'us_import3'
set_up_config(DIR)

# Normalise the numeric values
def normalize(val,_min,_max):
    return (val-_min)/(_max - _min)

df_train, df_test = fetch_data_sets()
for nc in NUMERIC_COLUMNS:
    _max = np.max(df_train[nc])
    _min = np.min(df_train[nc])
    df_train[nc] =  df_train[nc].parallel_apply(normalize, args=(_min,_max,))
    df_test[nc] =  df_test[nc].parallel_apply(normalize, args=(_min,_max,))

# ---------------------
# Generate anomalies from test set
# Count : 25% of the test set 
# ---------------------
for nc in NUMERIC_COLUMNS:
    numeric_col_stats[nc] = {}
    numeric_col_stats[nc]['median'] = np.median(df_train[nc])
    
create_coocc_dict(df_train) 

df_1 = df_test.sample(frac=0.25)
df_1 = df_1.reset_index(drop=True)
df_3 = df_1.parallel_apply(
    perturb_row,
    axis=1
)
# Save anomalies 
df_anomalies = df_3.copy()
df_anomalies.to_csv(os.path.join(save_dir,'gen_anomalies.csv'),index=False)

2118671
772048
110491
18456
58576
56038
2220610
5120156
733139
128272
399489
374255
14608460
264632
43218
141399
134051
5369725
6300
19805
19172
770584
3686
2690
133000
9415
416646
395415


IndexError: list index out of range